In [ ]:
USE TSQLV4;
GO
WITH EOY AS (
    SELECT *
    , DATEFROMPARTS(YEAR(orderdate), 12, 31) AS endofyear
    FROM Sales.Orders
)
SELECT orderid, orderdate, custid, empid, endofyear
FROM EOY
WHERE orderdate <> endofyear;


Proposition 1: Find average number of orders per customer using a subquarey then find those customers who have a lower than average order count.

1. Using a CTE we get the order count by counting the repeating <mark>OrderID</mark>'s in <mark>Sales.Orders</mark> then group by <mark>CustomerID</mark>
2. We find the case when the numbers we order counted is less than the average number of orders which we respectfully out put 'Less then avg' or 'Not less then avg'

Use Case:

> This is good to target customers who aren't ordering as many as the average and can be used for increasing overall sales.

In [ ]:
USE WideWorldImporters

WITH NOO AS (
    SELECT CustomerID, COUNT(OrderID) AS Numbers_of_orders
    FROM Sales.Orders
    GROUP BY CustomerID
)
SELECT CustomerID, Numbers_of_orders,
       CASE WHEN Numbers_of_orders < (
        SELECT AVG(Numbers_of_orders) 
        FROM NOO) 
        THEN 'Less then avg' 
        ELSE 'Not less then avg' 
        END AS LTA
FROM NOO


Proposition 2: List products with price above the average price of all products

1. Using a simple subquary we find the avg unit price and filter the ones above average.

Use case: This information helps identify if we are targeting a high income house hold or a lower house hold while also being able to see and adjust the overall retail pricings.

In [ ]:
USE WideWorldImporters

SELECT StockItemID, StockItemName, UnitPrice
FROM Warehouse.StockItems
WHERE UnitPrice > (
    SELECT AVG(UnitPrice) --Average is 44.16
    FROM Warehouse.StockItems 
)

Proposition 3: Find the latest order for each Customer

1. Using the MAX(<mark>OrderDate</mark>) we are able to find the most recent date.
2. Where clause to find only the <mark>OrderDate</mark>s IN the subquary 
3. Orderby DESC appropriately

Use case: Good to see in detail how the customers are ordering.

In [ ]:
USE WideWorldImporters

SELECT 
    CustomerID, 
    OrderID, 
    OrderDate
FROM Sales.Orders AS o
WHERE OrderDate IN (
    SELECT MAX(OrderDate)
    FROM Sales.Orders
    WHERE CustomerID = o.CustomerID
)
ORDER BY OrderDate DESC;

Proposition 4: List customers with their total number of orders using a CTE.

1. Using CTE we count the <mark>TotalOrders</mark> and group by <mark>CustomerID</mark> appropriately 
2. Join Customers and Order tables to find the appropriate name that match their ID

Use Case: The CTE is good globaly and can be used for other propositions.

In [ ]:
USE WideWorldImporters;

WITH CustomerOrders AS (
    SELECT 
        CustomerID, 
        COUNT(OrderID) AS TotalOrders
    FROM Sales.Orders
    GROUP BY CustomerID
)
SELECT 
    c.CustomerID,
    c.CustomerName,
    co.TotalOrders
FROM 
    Sales.Customers AS c
JOIN
    CustomerOrders AS co ON c.CustomerID = co.CustomerID

Propositon 5: Count of products supplied by each supplier using CTE

1. Find the amount of products supplied by each supplier by counting their ID in the stockItems table.
2. GroupBY SupplierID in the CTE

Use Case: Finds out which supplier is doing the most in terms of quantity per unique item.

In [ ]:
USE WideWorldImporters;

WITH SupplierCount AS (
    SELECT 
        SupplierID,
        COUNT(SupplierID) AS ProductCount
    FROM 
        Warehouse.StockItems
    GROUP BY 
        SupplierID
)
SELECT 
    SupplierID,
    ProductCount
FROM 
    SupplierCount;


Proposition 6: Find orders with more than the average number of orderlines

1. count the orderlineID's as orderlinecount and group by orderid
2. Use having instead of where because of the timing.
3. We compare the orderlinecounts that are greater than the avg count

Use Case: It is good to identify high volume orders

In [ ]:
USE WideWorldImporters;

SELECT OrderID, COUNT(OrderLineID) AS OrderLineCount
FROM Sales.OrderLines
GROUP BY OrderID
HAVING COUNT(OrderLineID) > (
    SELECT AVG(OrderLineCount)
    FROM (
        SELECT COUNT(OrderLineID) AS OrderLineCount
        FROM Sales.OrderLines
        GROUP BY OrderID
    ) AS AvgOrderLines
);


Proposition 7: Find the most expensive product per supplier using CTE

1. Find the most expensive item by the supplier grouping by supplierId
2. Then join stockitems and maxprices finding specifically the items that are the most expensive

Use Case: Finds the supplier that ships the most pricy item.

In [ ]:
USE WideWorldImporters;

WITH MaxPrices AS (
    SELECT 
        SupplierID,
        MAX(UnitPrice) AS MaxUnitPrice
    FROM 
        Warehouse.StockItems
    GROUP BY 
        SupplierID
)

SELECT 
    si.SupplierID, 
    si.StockItemID, 
    si.UnitPrice
FROM 
    Warehouse.StockItems AS si
JOIN 
    MaxPrices AS mp ON si.SupplierID = mp.SupplierID AND si.UnitPrice = mp.MaxUnitPrice;


Proposition 8: Find the total number of orders place by each customer

1. The CTE calculates the total number of orders for each customers
2. Left join the customers and orders to include all customers
3. count order id and group by the customerid and name

Use Case: The CTE is useful for other quary's and is good to know total number of orders per customer.

In [ ]:
USE WideWorldImporters;

WITH CustomerOrderCounts AS (
    SELECT 
        C.CustomerID, 
        C.CustomerName,
        COUNT(O.OrderID) AS TotalOrders
    FROM 
        Sales.Customers AS C
    LEFT JOIN 
        Sales.Orders AS O ON C.CustomerID = O.CustomerID
    GROUP BY 
        C.CustomerID, C.CustomerName
)

SELECT 
    CustomerID, 
    CustomerName, 
    TotalOrders
FROM 
    CustomerOrderCounts



<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Proposition 9: List the orders by customers in the last month</span>  

1.  inner quary joins both orderlines and order to access orderdate
2. filters the result to include only yhoser orders where the orderdate is within the last month.

Use case: Good to see what happened in the previous month

In [ ]:
USE WideWorldImporters;

SELECT 
    SI.StockItemID,
    SI.StockItemName
FROM 
    Warehouse.StockItems SI
WHERE 
    SI.StockItemID IN (
        SELECT OL.StockItemID
        FROM Sales.OrderLines OL
        JOIN Sales.Orders O ON OL.OrderID = O.OrderID
        WHERE O.OrderDate >= DATEADD(MONTH, -1, 2013-01-01 ) --cannot use current date such as sysdate so a random date form the database was added
    );


Proposition 10: Retrive the avg order size for each customer

1. Inside the cte find the total number of items for each order by sum(ol.quantity
2. combine order and order line with join
3. group by the cusomerid and orderid
4. calculate the avg number of items ordered per customer

Use case: good to have an idea of how many much people order

In [ ]:
USE WideWorldImporters;

WITH OrderSizes AS (
    SELECT 
        O.CustomerID,
        O.OrderID,
        SUM(OL.Quantity) AS TotalItems
    FROM 
        Sales.Orders AS O
    JOIN 
        Sales.OrderLines AS OL ON O.OrderID = OL.OrderID
    GROUP BY 
        O.CustomerID, O.OrderID
)

SELECT 
    CustomerID,
    AVG(TotalItems) AS AverageOrderSize
FROM 
    OrderSizes
GROUP BY 
    CustomerID;
